In [11]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys
import os

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
from src.rocket import Rocket

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

# Rocket setup
Ts = 1 / 20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)


print("Linearization around a steady state:")

xs, us = rocket.trim()
print("Default trim function:")
print("xs = ", xs)
print("us = ", us)

xs, us = rocket.trim()
print("xs = ", xs)
print("us = ", us)

x_ref = np.array([0.0] * 9 + [1.0, 0.0, 3.0])
xs, us = rocket.trim(x_ref)
print("x_ref = ", x_ref)
print("xs = ", xs)
print("us = ", us)

Linearization around a steady state:
Default trim function:
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
us =  [ 0.          0.         66.66666667  0.        ]
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
us =  [ 0.          0.         66.66666667  0.        ]
x_ref =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
xs =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
us =  [ 0.          0.         66.66666667  0.        ]


In [13]:
sys = rocket.linearize_sys(xs, us)
sys.info()  # see whole system


 System name: 'complete system'
------------------------------------------------------------
self.A shape: (12, 12)
self.B shape: (12, 4)
self.C shape: (12, 12)
self.D shape: (12, 4)
self.xs: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 3.]
self.us: [ 0.          0.         66.66666667  0.        ]

🔹 State variables:
        wx [rad/s]
        wy [rad/s]
        wz [rad/s]
     alpha [rad]
      beta [rad]
     gamma [rad]
        vx [m/s]
        vy [m/s]
        vz [m/s]
         x [m]
         y [m]
         z [m]
→ access via self.state_name, self.state_unit

🔹 Input variables:
        d1 [rad]
        d2 [rad]
      Pavg [%]
     Pdiff [%]
→ access via self.input_name, self.input_unit

🔹 Output variables:
        wx [rad/s]
        wy [rad/s]
        wz [rad/s]
     alpha [rad]
      beta [rad]
     gamma [rad]
        vx [m/s]
        vy [m/s]
        vz [m/s]
         x [m]
         y [m]
         z [m]
→ access via self.output_name, self.output_unit
-------------------------------------

In [14]:
sys_x, sys_y, sys_z, sys_roll = sys.decompose()
sys_z.info()


 System name: 'sys_z'
------------------------------------------------------------
self.A shape: (2, 2)
self.B shape: (2, 1)
self.C shape: (1, 2)
self.D shape: (1, 1)
self.xs: [0. 3.]
self.us: [66.66666667]
self.idx: [ 8 11]
self.idu: [2]
self.idy: [11]

🔹 State variables:
        vz [m/s]
         z [m]
→ access via self.state_name, self.state_unit

🔹 Input variables:
      Pavg [%]
→ access via self.input_name, self.input_unit

🔹 Output variables:
         z [m]
→ access via self.output_name, self.output_unit
------------------------------------------------------------
 To check data directly, try:
   sys.A, sys.B, sys.C, sys.D
   sys.xs, sys.us, sys.state_name, etc.
------------------------------------------------------------


In [15]:
print(sys_x.A, sys_x.B, sys_x.C, sys_x.D)
print(sys_y.A, sys_y.B, sys_y.C, sys_y.D)
print(sys_z.A, sys_z.B, sys_z.C, sys_z.D)
print(sys_roll.A, sys_roll.B, sys_roll.C, sys_roll.D)

[[0.    0.    0.    0.   ]
 [1.    0.    0.    0.   ]
 [0.    9.806 0.    0.   ]
 [0.    0.    1.    0.   ]] [[-65.47484472]
 [  0.        ]
 [  9.806     ]
 [  0.        ]] [[0. 0. 0. 1.]] [[0.]]
[[ 0.     0.     0.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.    -9.806  0.     0.   ]
 [ 0.     0.     1.     0.   ]] [[-65.47484472]
 [  0.        ]
 [ -9.806     ]
 [  0.        ]] [[0. 0. 0. 1.]] [[0.]]
[[0. 0.]
 [1. 0.]] [[0.14709]
 [0.     ]] [[0. 1.]] [[0.]]
[[0. 0.]
 [1. 0.]] [[0.104]
 [0.   ]] [[0. 1.]] [[0.]]


DELIVRABLE 2.1 

The states affected by an specific input are not afffcted by any other.

Threfore, since we use a linearised system we can apply the superposition principle.

Servo motor 1 affects $\delta _{1}$ which affects angular speed $\omega _y$ and position $x$ and velocity $v_x$

Servo motor 2 affects $\delta _{2}$ which affects angular speed $\omega _x$ and position $y$ and velocity $v_y$

Average throttle $P_{avg}$ which affects position and velocity $v_z$

Differential throttle $P_{diff}$ affects the roll angle $\gamma$ and angular speed $\omega _z$.

In [ ]:
Ad, Bd, Cd = sys_x._discretize(Ts)
print(Ad, Bd, Cd)

[[ 1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [ 5.00000000e-02  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.22575000e-02  4.90300000e-01  1.00000000e+00  0.00000000e+00]
 [ 2.04291667e-04  1.22575000e-02  5.00000000e-02  1.00000000e+00]] [[-3.27374224]
 [-0.08184356]
 [ 0.47692403]
 [ 0.0120903 ]] [[0. 0. 0. 1.]]


In [17]:
[Ad, Bd, Cd] = sys_y._discretize(Ts)
Ad, Bd, Cd

(array([[ 1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00],
        [ 5.00000000e-02,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.22575000e-02, -4.90300000e-01,  1.00000000e+00,
          0.00000000e+00],
        [-2.04291667e-04, -1.22575000e-02,  5.00000000e-02,
          1.00000000e+00]]),
 array([[-3.27374224],
        [-0.08184356],
        [-0.47692403],
        [-0.0120903 ]]),
 array([[0., 0., 0., 1.]]))

In [18]:
[Ad, Bd, Cd] = sys_z._discretize(Ts)
Ad, Bd, Cd

(array([[1.  , 0.  ],
        [0.05, 1.  ]]),
 array([[0.0073545 ],
        [0.00018386]]),
 array([[0., 1.]]))

In [19]:
[Ad, Bd, Cd] = sys_roll._discretize(Ts)
Ad, Bd, Cd

(array([[1.  , 0.  ],
        [0.05, 1.  ]]),
 array([[0.0052 ],
        [0.00013]]),
 array([[0., 1.]]))